In [1]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import random

In [2]:
Types = {'electrique': 6210 , 'thermique':3740 } #le taux de co2 emis a la construction en kg 

CO2emisparkm = {'electrique': 0.02 , 'thermique':0.19 } #le taux de co2 emis par km en kg 

Types_vitesseslimit = {'electrique': 200, 'thermique':150}

Types_perc = {'electrique': 0.5, 'thermique':0.5}

Proba_shortestpath = 0.8 #la proba de choisir le chemin le plus court

In [3]:
class Voiture:
    """ Création d'une voiture avec ses différents parametres"""
    nbr_voiture = 0
    def __init__(self,typee,positiondepart,positionarrivé,positionactuelle,vitesse,vitesse_max,villeg):
        self.type = typee
        self.speed = vitesse 
        """position est un tuple d'un tuple de position et d'un pourcentage """
        self.position = positionactuelle
        self.positiondepart = positiondepart
        self.positionarrivé = positionarrivé
        self.vitesse_max =vitesse_max
        "chemin que va prendre une voiture "
        self.chemin = path_choice (Proba_shortestpath,villeg,positiondepart, positionarrivé)
        self.CO2=0.0
        self.stock=0.0
        Voiture.nbr_voiture += 1
    def spawn(typesperc,nbr,lenvillem,villeg):
        """ création de la population de voitures en prenant en compte le pourcentage de voitures éléctriques et terminques"""
        system=[]
        len_ville_1m = lenvillem
        for i in range(round(nbr*typesperc['electrique'])):
            
            pos = random_position(len_ville_1m)
            depart = pos[0]
            arrivee = pos[1]
            
            
            a = path_choice (Proba_shortestpath,villeg,depart, arrivee)
            
            system.append("voiture"+str(i+1))
            system[i]= Voiture('electrique',depart,arrivee,((depart,a[1]),0.0),50,Types_vitesseslimit['electrique'],villeg)
            system[i].CO2=CO2emisparkm['electrique']
        
        for j in range(round(nbr*typesperc['thermique']), nbr):
            
            pos = random_position(len_ville_1m)
            depart = pos[0]
            arrivee = pos[1]
            
            b = path_choice (Proba_shortestpath,villeg,depart, arrivee)
        
            system.append("voiture"+str(j+1))
            system[j]= Voiture('thermique',depart,arrivee,((depart,b[1]),0.0),50,Types_vitesseslimit['thermique'],villeg)
            system[j].CO2=CO2emisparkm['thermique']
        return system

In [4]:
def co2_emission(distance,typee):
    "prend la distance en km et le type de la voiture et renvoie le taux de co2 emis"
    return distance * CO2emisparkm[typee]

In [5]:
def co2_construction_liste (liste_voiture):
    """renvoie le taux de co2 cumulé emis lors de la construction des voitures de différents type de la liste de voiture sur 
    forme de tuples """
    ther=0.0
    elec=0.0
    for i in range(len(liste_voiture)):
        if liste_voiture[i].type=='electrique':
            elec=elec+Type['electrique']
    for j in range(len(liste_voiture)):
        if liste_voiture[j].type=='thermique':
            ther=ther+Type['thermique']
    return (ther,elec)

In [ ]:
def change_distance(voiture,verbose=False):
    """ Cette fonction permet de deplacer une voiture pendant 10s. Elle modifie les parametres de la voiture :
    pourcentage de la distance parcouru sur une arrete, changement de node quand on arrive à la fin de l'arrete
    et ajout de la distance parcouru sur la nouvelle arrete"""
    
    chemin_liste = voiture.chemin
    chemin = liste2tuple(chemin_liste)
    
    distance_parcouru =(voiture.speed*10)/3600
    if voiture.position[1]+(distance_parcouru*100)/donne_longueur(voiture.position[0]) > 100:
        if voiture.position[0][1]== voiture.positionarrivé:
            voiture.speed=0
            voiture.position=(voiture.position[0],100)
        else:
            if verbose:
                print()
                print("changement d'arrete " + "passage de " +str(chemin[indice_chemin(chemin,voiture.position[0]) - 1]) + " à :" + str(voiture.position[0]))
                print()
            voiture.position = (chemin[indice_chemin(chemin,voiture.position[0]) + 1],0)
        
    
    if voiture.position[1]<100 and voiture.speed!= 0:
        voiture.position = (voiture.position[0],voiture.position[1]+(distance_parcouru*100)/donne_longueur(voiture.position[0])) 
        voiture.stock += co2_emission(voiture.speed*10/3600,voiture.type)

In [6]:
def co2_voiture_graph(lv):
    """ Affiche le graphe des emissions de co2 pour les voitures thermiques et electriques """
    ther=0.0
    elec=0.0
    for i in range(len(lv)):
        if lv[i].type=='electrique':
            elec=elec+lv[i].stock
    for j in range(len(lv)):
        if lv[j].type=='thermique':
            ther=ther+lv[j].stock
    """Pour la partie avec plt je ne sais pas comment faire pour afficher les barres de manière
    exacte en plus je ne comprend rien à vos maneuvre sur cette partie donc je met tout en commentaire """
    """abscisse=["thermique","electrique"]
    ordonnee=[ther,elec]
    plt.bar(abscisse,ordonnee)"""